In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## basic imports
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

# sklearn imports
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from category_encoders import TargetEncoder, LeaveOneOutEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import StratifiedShuffleSplit,train_test_split

# Keras imports 
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils



In [ ]:
# loading data and getting shape
df_train = pd.read_csv("../input/cat-in-the-dat/train.csv") 
df_test = pd.read_csv("../input/cat-in-the-dat/test.csv")
df_sample_submission = pd.read_csv("../input/cat-in-the-dat/sample_submission.csv")
print(df_train.shape, df_test.shape, df_sample_submission.shape)


In [ ]:
## analysing the binary features
binary = [ 'bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']
for i in binary:
    print(df_train[i].value_counts())


In [ ]:
nominal = ['nom_0', 'nom_1','nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
for i in nominal:
    print(f'{i}: {df_train[i].value_counts()}')

In [ ]:
## Do not worth work with 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9' because the processing will be very hard
nominal = ['nom_0', 'nom_1','nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']
for i in nominal:
    print(f'The diferent values in {i} feature is: {df_train[i].value_counts().count()}')

In [ ]:
ordinal = ['ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month']
for i in ordinal:
    print(f'{i}: {df_train[i].value_counts()}')

In [ ]:
## Do not worth work with 'ord_3', 'ord_4', 'ord_5' because the processing will be very hard
ordinal = ['ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5', 'day', 'month']
for i in ordinal:
    print(f'The diferent values in {i} feature is: {df_train[i].value_counts().count()}')

In [ ]:
## Slicing for validate the best model 
## using only 5000 lines to validate ... to spend my time
X = df_train.drop(['id', 'target'], axis = 1).head(10000)
X_teste= df_test.drop('id', axis = 1).head(10000)
y = df_train['target'].values


In [ ]:
X.shape, X_teste.shape

In [ ]:
## let's transform to categorical data
columns = ['bin_3', 'bin_4', 'nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4','nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'ord_0','ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']
for i in columns:
    X[i] = pd.Categorical(X[i]).codes
    X_teste[i] = pd.Categorical(X_teste[i]).codes
    
X = X.values
X_teste = X_teste.values    

ohe = OneHotEncoder(categorical_features=[5, 6,7,8,9])
X = ohe.fit_transform(X).toarray()
X_teste = ohe.fit_transform(X_teste).toarray()

In [ ]:
X.shape, X_teste.shape

In [ ]:
%%time
## Trying to validade the model here... testing with some algoritms and verifying the best perform
kfold = StratifiedKFold(n_splits=5, shuffle = True, random_state=0)
modelos = {'Logistic': LogisticRegression(solver = 'lbfgs', C =  0.1, max_iter = 5000), 'naive': GaussianNB(), 'random': RandomForestClassifier(n_estimators=100), 'tree': DecisionTreeClassifier(), 'knn': KNeighborsClassifier(), 'svc': SVC(gamma='scale'),
          'neural': MLPClassifier(learning_rate='adaptive', max_iter=2000)}
resultados = {}
desvio_padrao = {}
matrizes = []
for i, j in modelos.items():
    for ind_treinamento, ind_teste in kfold.split(X, np.zeros(shape = (X.shape[0], 1))):
        modelo = j
        modelo.fit(X[ind_treinamento], y[ind_treinamento])
        previsoes = modelo.predict(X[ind_teste])
        precisao = accuracy_score(y[ind_teste], previsoes)
        matrizes.append(confusion_matrix(y[ind_teste], previsoes))        
        resultados[i] = np.mean(precisao)
        desvio_padrao[i] =  np.std(precisao)
    matriz_final = np.mean(matrizes, axis = 0)
    print(f'Resultados para o modelo {i}: {resultados[i]}')
    print(f'Desvio padrão para o modelo {i}: {desvio_padrao[i]}')
    print(matriz_final)

In [ ]:
## analysing better approach
## not good results until here and the processing is very hard
sns.barplot(list(modelos.keys()), list(resultados.values()))

plt.grid()

In [ ]:
modelo_svc = SVC(gamma='scale', verbo)
modelo_svc.fit(X, y)
predicao = modelo_svc.predict(X_teste)
df_submission = pd.DataFrame({'id': df_test['id'], 'target': predicao})
df_submission.to_csv('predict_svc.csv', index = False)

## Result: 0.60 (not very good)

In [ ]:
modelo_random = RandomForestClassifier(n_estimators=100,  verbose = True)
modelo_random.fit(X, y)
predicao = modelo_random.predict(X_teste)
df_submission = pd.DataFrame({'id': df_test['id'], 'target': predicao})
df_submission.to_csv('predict_random.csv', index = False)

## Result 0.62 (not very good too)



# **Let's try another approach**

In [ ]:
# Organizing data
target = df_train['target']
train_id = df_train['id']
test_id = df_test['id']
df_train.drop(['target', 'id'], axis=1, inplace=True)
df_test.drop('id', axis=1, inplace=True)

print(df_train.shape)
print(df_test.shape)

In [ ]:
%%time

# using Dummies variables in the model and expand a lot the dimensions  - sparse matrix
traintest = pd.concat([df_train, df_test])
dummies = pd.get_dummies(traintest, columns=traintest.columns, drop_first=True, sparse=True)
df_train_ohe = dummies.iloc[:df_train.shape[0], :]
df_test_ohe = dummies.iloc[df_train.shape[0]:, :]

print(df_train_ohe.shape)
print(df_test_ohe.shape)

In [ ]:
dummies.shape

In [ ]:
%%time
## using tocoo() to transform in coordinate format and tocsr() to compress matrix
X = df_train_ohe.sparse.to_coo().tocsr()
X_teste = df_test_ohe.sparse.to_coo().tocsr()
y = target.values


In [ ]:
X, X_teste

In [ ]:
%%time
## functions to create the models - LogisticRegression and XgbootClassifier

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score as auc
from sklearn.linear_model import LogisticRegression

# Model
def run_cv_model(train, test, target, model_fn, params={}, eval_fn=None, label='model'):
    kf = KFold(n_splits=5)
    fold_splits = kf.split(train, target)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros((train.shape[0]))
    i = 1
    for dev_index, val_index in fold_splits:
        print('Started ' + label + ' fold ' + str(i) + '/5')
        dev_X, val_X = train[dev_index], train[val_index]
        dev_y, val_y = target[dev_index], target[val_index]
        params2 = params.copy()
        pred_val_y, pred_test_y = model_fn(dev_X, dev_y, val_X, val_y, test, params2)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        if eval_fn is not None:
            cv_score = eval_fn(val_y, pred_val_y)
            cv_scores.append(cv_score)
            print(label + ' cv score {}: {}'.format(i, cv_score))
        i += 1
    print('{} cv scores : {}'.format(label, cv_scores))
    print('{} cv mean score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv std score : {}'.format(label, np.std(cv_scores)))
    pred_full_test = pred_full_test / 5.0
    results = {'label': label,
              'train': pred_train, 'test': pred_full_test,
              'cv': cv_scores}
    return results


def runLR(train_X, train_y, test_X, test_y, test_X2, params):
    print('Train LR')
    model = LogisticRegression(**params)
    model.fit(train_X, train_y)
    print('Predict 1/2')
    pred_test_y = model.predict_proba(test_X)[:, 1]
    print('Predict 2/2')
    pred_test_y2 = model.predict_proba(test_X2)[:, 1]
    return pred_test_y, pred_test_y2


def runXGB(train_X, train_y, test_X, test_y, test_X2, params):
    print('Train XGB')
    model = XGBoostClassifier()
    model.fit(train_X, train_y)
    print('Predict 1/2')
    pred_test_y = model.predict_proba(test_X)[:, 1]
    print('Predict 2/2')
    pred_test_y2 = model.predict_proba(test_X2)[:, 1]
    return pred_test_y, pred_test_y2

### You can change the function when you call the function. In this case I call using runLR

lr_params = {'solver': 'lbfgs', 'C': 0.1}
results = run_cv_model(X, X_teste, target, runXGB, lr_params, auc, 'lr')

In [ ]:
modelo_logistic = LogisticRegression(solver = 'lbfgs', C=0.1)
modelo_logistic.fit(X, y)
predicao = modelo_logistic.predict(X_teste)
df_submission = pd.DataFrame({'id': test_id, 'target': predicao})
df_submission.to_csv('predict_logistic.csv', index = False)

## Result 0.80739 with LogisticRegression